## Assignment - A4.part2
### Devin Hall, Tyler Birnie
### Due: 4-3-22 @ 11:59pm
Some pre-requisites we will need for this project

### Question 1: NBA Data

#### Question 1A: 
Load the data of NAB plays in the 2018-2019 season.
First, filter the players to only consider those who have played in more than 20 games. The analysis will ignore the first 7 columns as well as ignore the columns of statistics of prefectages (FG%, 3P%, 2P%,eFG%,FT%)

#### Question 1B:
The features have different ranges, therefore we should scale the date before considering the clustering analysis. Scal the data using min-max normalization with range of [0,1]

Python- MinMaxScaler from sklearn.preprocessing

#### Question 1C:
Run Kmeans clustering on the data with k=2, ..., 10. For each value of k, keep track of the within-cluster variation. This quantity is referred to as different terms such as "inertia" and total "within-cluster sum-of-squares"

Python - KMeans from sklearn.cluster.

Plot the within-cluster variation vs the values of k.
As discussed thi value is alwasys decresasing, therefore it is difficult to use for selecting the _best_ value of k.

#### Question 1D:
Determine the "best" number ofclusters using gap statistic. 
Python- gapstat.py available on Canvas with B1=50.

#### Question 1E: 
Create a data frame with the mean skill values (centers) of each group, using the best number of groups determined in (d), as a table or data frame. 

You may want to run Kmeans again with the best value of k.

Print out the statistics for each group (rows) and columns of "MP',"FG","3P", and "FT".

#### Question 1F:
Report the same statistics as in (e), but using the original data scaling (reverse the scalling back to the original data range).

#### Question 1G:
Apply PCA to the filtered nba data (make sure to apply the necessary scaling for running PCA). Plot the data in the first two principal components, coloed by the best group lables found in (e).